## Load data

In [63]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
random_state = 12041500

In [64]:
def load_data():
    df_train = pd.read_json("./data/synthetic_data_CTGANSynthesizer.json")
    df_test = pd.read_json("./data/testset.json")

    df_train.drop(columns=['fnlwgt'], inplace=True)
    df_test.drop(columns=['fnlwgt'], inplace=True)

    return df_train, df_test

df_train, df_test = load_data()

ratio_features = ["age", "capital-gain", "capital-loss", "hours-per-week"]
ordinal_features = ["education-num"]
nominal_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
target = 'income'

## Train baseline

In [65]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [66]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [67]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8361032716630082
Precision      0.7822701965699763
Recall         0.7458442654631545
F1             0.7608356769787064


## Fairness Evaluation

In [68]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [69]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [70]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [71]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [72]:
print(privileged_subset)

({'capital-gain': [0, 1, 4, 5, 6]}, 226.8525)


In [73]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-gain']

                         Group Distance  Proportion  Counts   P-Value
capital-gain [37.00, 16383.00]    0.434    0.095449    3563  0.00e+00
    capital-gain [-0.00, 5.00]   -0.137    0.517078   19302 1.48e-323
   capital-gain [23.00, 37.00]    0.253    0.097458    3638 3.41e-256
   capital-gain [15.00, 23.00]    0.087    0.106352    3970  1.82e-38
    capital-gain [5.00, 10.00]   -0.047    0.097645    3645  2.44e-13

Weighted Mean Statistical Distance: 0.1648489023109496


In [74]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 19340, we observe 0.0639 as the average probability of earning >50k, but our model predicts 0.2058


## Fairness Metrics

In [75]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [76]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

1744 Na rows removed!
202 Na rows removed!


In [77]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.2197822046732668
average_odds_difference         0.18205092550757615
equal_opportunity_difference    0.29737495102520567
disparate_impact                5.9488519842526815
theil_index                     0.11351788790247734


In [78]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [79]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,0.157790,0.301264,0.548028,80.920614,0.113518
1e-10,0.176903,0.143317,0.245121,4.778996,0.113518
0.001,-0.072405,-0.190094,-0.251483,0.678597,0.113518
0.01,0.106165,0.276341,0.548028,3.123298,0.113518
0.1,0.164673,0.148689,0.262768,4.440570,0.113518
0,0.157790,0.301264,0.548028,80.920614,0.113518
1,0.219782,0.182051,0.297375,5.948852,0.113518
5,0.208665,0.180456,0.297944,6.022775,0.113518
10,0.201300,0.189152,0.317138,6.455366,0.113518
25,0.201300,0.189152,0.317138,6.455366,0.113518


In [80]:
df_fairness_metrics.to_json("./results/fairness_metrics_synthetic.json")

In [81]:
def calc_diff(data, expectations, target, subset):
    if len(subset) == 0:
        return {
            "count": 0,
            "expected_probability": np.NaN,
            "model_probability": np.NaN
        }

    _df = data.copy()
    _df["expectations"] = expectations.copy()

    _to_choose = _df[subset.keys()].isin(subset).all(axis=1)
    _to_choose = _df.loc[_to_choose]

    return {
        "count": len(_to_choose),
        "expected_probability": np.round(_to_choose[target].mean(), 4),
        "model_probability": np.round(_to_choose["expectations"].mean(), 4),
    }

In [82]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

df_probs_diff = pd.DataFrame(
    columns=[
        "count",
        "expected_probability",
        "model_probability",
    ]
)
for penalty, priv in priviliged_subsets.items():
    df_probs_diff.loc[penalty] = calc_diff(df_train, probs, target, priv[0])

In [83]:
df_probs_diff

,count,expected_probability,model_probability
1e-17,1013,0.0000,0.1981
1e-10,14398,0.0497,0.2066
0.001,1345,0.0766,0.2172
0.01,240,0.0000,0.1864
0.1,12933,0.0496,0.2051
0,1013,0.0000,0.1981
1,19340,0.0639,0.2058
5,17732,0.0594,0.2059
10,16030,0.0546,0.2068
25,16030,0.0546,0.2068


## Binned Data

In [84]:
df_train, df_test = load_data()
nominal_features = nominal_features + ['age', 'hours-per-week', 'capital-gain', 'capital-loss']

In [85]:
import numpy as np
import pandas as pd

def clean_data(X: pd.DataFrame) -> pd.DataFrame:
    X = X.reset_index(drop=True)
    cols = list(X.columns)
    X[cols] = X[cols].replace([" ?"], np.nan)
    X = X.dropna()
    def strip_str(x):
        if isinstance(x, str):
            return x.strip()
        else:
            return x
    X = X.applymap(strip_str)
    # X["relationship"] = X["relationship"].replace(["Husband", "Wife"], "Married")
    X["hours-per-week"] = pd.cut(
        x=X["hours-per-week"],
        bins=[0.9, 25, 39, 40, 55, 100],
        labels=["PartTime", "MidTime", "FullTime", "OverTime", "BrainDrain"],
    )
    X.age = pd.qcut(X.age, q=5)
    X["capital-gain"] = pd.cut(
        x=X["capital-gain"],
        bins=[-1, 0, 5000, 10000, 50000, 100000],
        labels=["NoGain", "LowGain", "MediumGain", "HighGain", "VeryHighGain"],
    )
    X["capital-loss"] = pd.cut(
        x=X["capital-loss"],
        bins=[-1, 0, 1000, 2000, 5000, 100000],
        labels=["NoLoss", "LowLoss", "MediumLoss", "HighLoss", "VeryHighLoss"],
    )

    return X

df_train = clean_data(df_train.dropna())
df_test = clean_data(df_test.dropna())

In [86]:
for col in df_train.columns:
    if df_train[col].dtype == "category":
        df_train[col] = df_train[col].astype("object")
        df_test[col] = df_test[col].astype("object")

### Train baseline

In [87]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [88]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [89]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8366259015365318
Precision      0.815639353554005
Recall         0.7010494485756561
F1             0.7318629810496294


### Fairness Evaluation

In [90]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [91]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [92]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [93]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [94]:
print(privileged_subset)

({'education-num': [11, 13, 14], 'capital-loss': ['NoLoss'], 'sex': ['Male'], 'race': ['Black'], 'occupation': ['Prof-specialty', 'Protective-serv']}, 7.357)


In [95]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-loss', 'education-num', 'occupation', 'race', 'sex']

                               Group Distance  Proportion  Counts   P-Value
        Exec-managerial, White, Male    0.486    0.042460    1585  0.00e+00
         Prof-specialty, White, Male    0.373    0.061052    2279  0.00e+00
                     Exec-managerial    0.409    0.067240    2510  0.00e+00
                Prof-specialty, Male    0.348    0.078464    2929  0.00e+00
               Exec-managerial, Male    0.482    0.051140    1909  0.00e+00
       NoLoss, Exec-managerial, Male    0.472    0.039326    1468  0.00e+00
              Exec-managerial, White    0.417    0.054971    2052  0.00e+00
                              Female   -0.118    0.371481   13867 2.38e-317
             NoLoss, Exec-managerial    0.396    0.051381    1918 2.51e-312
NoLoss, Exec-managerial, White, Male    0.479    0.032495    1213 3.98e-283

Weighted Mean Statistical Distance: 0.13175298419252776


In [96]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 110, we observe 0.0455 as the average probability of earning >50k, but our model predicts 0.2014


### Fairness Metrics

In [97]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [98]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

In [99]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.027316204577818276
average_odds_difference         -0.22859767869483066
equal_opportunity_difference    -0.41778127458693937
disparate_impact                1.2003188335706674
theil_index                     0.10603210355698797


In [100]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [101]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,0.120956,0.297646,0.582492,3.797097,0.106032
1e-10,0.152072,0.313252,0.582492,13.469931,0.106032
0.001,0.152072,0.313252,0.582492,13.469931,0.106032
0.01,0.151993,0.313214,0.582492,13.387437,0.106032
0.1,0.161054,0.317384,0.582492,36.995486,0.106032
0,0.167404,0.320067,0.583104,178.281059,0.106032
1,0.027316,-0.228598,-0.417781,1.200319,0.106032
5,0.000000,0.000000,0.000000,1.000000,0.000000
10,0.000000,0.000000,0.000000,1.000000,0.000000
25,0.000000,0.000000,0.000000,1.000000,0.000000


In [102]:
df_fairness_metrics.to_json("./results/fairness_metrics_synthetic_cleaned.json")

In [103]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

df_probs_diff = pd.DataFrame(
    columns=[
        "count",
        "expected_probability",
        "model_probability",
    ]
)
for penalty, priv in priviliged_subsets.items():
    df_probs_diff.loc[penalty] = calc_diff(df_train, probs, target, priv[0])

In [104]:
df_probs_diff

,count,expected_probability,model_probability
1e-17,185,0.0000,0.0613
1e-10,164,0.0000,0.0929
0.001,164,0.0000,0.0929
0.01,163,0.0000,0.0935
0.1,447,0.0000,0.0404
0,1059,0.0076,0.0320
1,110,0.0455,0.2014
5,0,NaN,NaN
10,0,NaN,NaN
25,0,NaN,NaN


## PDFed Data

In [105]:
df_train, df_test = load_data()
nominal_features = nominal_features + ['age', 'hours-per-week', 'capital-gain', 'capital-loss']

In [106]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

def clean_data(X: pd.DataFrame) -> pd.DataFrame:
     # Function to transform numerical attributes into their PDF values
    def transform_to_pdf(data, numerical_columns, target):
        transformed_data = data.copy()
        for column in numerical_columns:
            if column not in ['sex', target]:
                kde = gaussian_kde(data[column].dropna())
                transformed_data[column] = kde(data[column])
        return transformed_data

    # Transforming the numerical attributes
    # X = X.drop(columns=["fnlwgt", "education"], errors="ignore")
    cols = list(X.columns)
    X[cols] = X[cols].replace([" ?"], np.nan)
    X = X.dropna()

    def strip_str(x):
        if isinstance(x, str):
            return x.strip()
        else:
            return x
    X = X.applymap(strip_str)

    # X["relationship"] = X["relationship"].replace(["Husband", "Wife"], "Married")
    X = transform_to_pdf(X,['age', 'hours-per-week', 'capital-gain', 'capital-loss'], 'income')
    return X

df_train = clean_data(df_train.dropna())
df_test = clean_data(df_test.dropna())

In [107]:
for col in df_train.columns:
    if df_train[col].dtype == "category":
        df_train[col] = df_train[col].astype("object")
        df_test[col] = df_test[col].astype("object")

### Train baseline

In [108]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [109]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [110]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.7481969269363437
Precision      0.7134257748776509
Recall         0.7852284631210857
F1             0.7161181202405134


### Fairness Evaluation

In [111]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [112]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [113]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [114]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [115]:
print(privileged_subset)

({'capital-gain': [0.0014268116068545864, 0.0014494447135108503, 0.0014499044018283546, 0.0014502686425889616, 0.0014506540370765706, 0.0014508448770105507], 'workclass': ['?', 'Local-gov'], 'age': [0.0047791226256496434, 0.006711487346189525, 0.006959676452913946, 0.0074541789835679915, 0.009036480031066975, 0.03695082496929242, 0.03792455714871646]}, 11.238)


In [116]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['age', 'capital-gain', 'workclass']

                                       Group Distance  Proportion  Counts   P-Value
                  capital-gain [-0.00, 0.00]    0.435    0.100029    3734  0.00e+00
         capital-gain [-0.00, 0.00], Private    0.431    0.061963    2313  0.00e+00
age [0.01, 0.02], capital-gain [-0.00, 0.00]    0.548    0.014921     557 2.95e-169
age [0.02, 0.03], capital-gain [-0.00, 0.00]    0.548    0.014278     533 4.20e-162
                                Self-emp-inc    0.338    0.032629    1218 1.36e-148
    capital-gain [-0.00, 0.00], Self-emp-inc    0.605    0.010099     377 1.41e-139
                capital-gain [0.00, 0.00], ?   -0.161    0.069865    2608 3.48e-126
age [0.01, 0.01], capital-gain [-0.00, 0.00]    0.497    0.013100     489 1.35e-123
                                           ?   -0.155    0.073026    2726 1.26e-119
                   capital-gain [0.00, 0.00]   -0.048    0.899971   33595 8.34e-114

Weighted Mean S

In [117]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 374, we observe 0.0 as the average probability of earning >50k, but our model predicts 0.2065


### Fairness Metrics

In [118]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [119]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

In [120]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.15056525605285226
average_odds_difference         0.32627874937004087
equal_opportunity_difference    0.6199711702267069
disparate_impact                10.385234293961124
theil_index                     0.0954776007468658


In [121]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [122]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,0.093033,-0.200346,-0.380079,2.280574,0.095478
1e-10,0.152394,0.326953,0.619971,11.229462,0.095478
0.001,0.103443,0.302980,0.619971,2.664494,0.095478
0.01,0.115892,0.309269,0.619971,3.334394,0.095478
0.1,0.086654,0.294659,0.619971,2.100508,0.095478
0,0.093033,-0.200346,-0.380079,2.280574,0.095478
1,0.150565,0.326279,0.619971,10.385234,0.095478
5,0.000000,0.000000,0.000000,1.000000,0.000000
10,0.000000,0.000000,0.000000,1.000000,0.000000
25,0.000000,0.000000,0.000000,1.000000,0.000000


In [123]:
df_fairness_metrics.to_json("./results/fairness_metrics_synthetic_pdf.json")

In [124]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

df_probs_diff = pd.DataFrame(
    columns=[
        "count",
        "expected_probability",
        "model_probability",
    ]
)
for penalty, priv in priviliged_subsets.items():
    df_probs_diff.loc[penalty] = calc_diff(df_train, probs, target, priv[0])

In [125]:
df_probs_diff

,count,expected_probability,model_probability
1e-17,234,0.0043,0.1892
1e-10,537,0.0000,0.1921
0.001,177,0.0000,0.1911
0.01,141,0.0000,0.2286
0.1,127,0.0000,0.1930
0,234,0.0043,0.1892
1,374,0.0000,0.2065
5,0,NaN,NaN
10,0,NaN,NaN
25,0,NaN,NaN
